In [1]:
import cv2 as cv
import numpy as np 
from matplotlib import pyplot as plt
import serial
import time

In [ ]:
Arduino = serial.Serial('COM4',9600)
time.sleep(2)

vid = cv.VideoCapture(0)

lower_yellow = np.array([0,125,125])
upper_yellow = np.array([10,255,255])

while(vid.isOpened()):
    
    #capture frame by frame 
    ret,frame = vid.read()
    frame = cv.flip(frame,1) 
    # 2nd : determines the axis around which to be flipped 0: x axis , 1 : y axis , -1 both axis 
    
    w = frame.shape[1]
    h = frame.shape[0]
    
    image_smooth = cv.GaussianBlur(frame,(7,7),0)
    
    # defining the region of interest
    mask = np.zeros_like(frame)
    mask[50:350 , 50:350] = [255,255,255]
    image_roi = cv.bitwise_and(image_smooth,mask)
    cv.rectangle(frame ,(50,50),(350,350),(0,0,255),2)
    cv.line(frame ,(150,50),(150,350),(0,0,255),1)
    cv.line(frame ,(250,50),(250,350),(0,0,255),1)
    cv.line(frame ,(50,150),(350,150),(0,0,255),1)
    cv.line(frame ,(50,250),(350,250),(0,0,255),1)
    
    
    # threshold the image for the yellow
    image_hsv = cv.cvtColor(image_roi,cv.COLOR_BGR2HSV)
    image_threshold = cv.inRange(image_hsv,lower_yellow,upper_yellow)
    # 1 for with in the bound else 0
    
    # find contour 
    contours,hierachy = cv.findContours(image_threshold,cv.RETR_TREE,cv.CHAIN_APPROX_NONE)
    
    #find the index of the largest contour
    if(len(contours)!=0):
        areas = [cv.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        cnt = contours[max_index]
        
        
        #pointer on video
        M= cv.moments(cnt)
        if(M['m00']!=0):
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            cv.circle(frame,(cx,cy),10,(0,255,0),-1)
            
            #cursor motion
            if cx in range(150,250):
                if cy <150:
                    Arduino.write(b'f')
                    print("Forward")
                    
                elif cy > 250:
                    Arduino.write(b'b')
                    print("Backward")
                    
                else:
                    Arduino.write(b's')
                    print("Stop")
            if cy in range(150,250):
                if cx <150:
                    Arduino.write(b'l')
                    print("left")
                    
                elif cx > 250:
                    Arduino.write(b'r')
                    print("right")
                    
                else:
                    Arduino.write(b's')
                    print("Stop")
        
    # Display the resulting frame 
    cv.imshow('frame',frame)
    
    key = cv.waitKey(100)
    if key ==27:
        break

vid.release()
cv.destroyAllWindows()